In [1]:
# get datasetd from "geometry of truth paper"
!git clone git@github.com:saprmarks/geometry-of-truth.git /root/geometry-of-truth
!mv /root/geometry-of-truth/datasets/*.csv data

fatal: destination path '/root/geometry-of-truth' already exists and is not an empty directory.
mv: cannot stat '/root/geometry-of-truth/datasets/*.csv': No such file or directory


In [2]:
import os
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
from baukit import Trace
import torch
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [3]:
# import my modules
import importlib
# join the path to the modules to the current working directory

import utils, dataset_utils
importlib.reload(utils)
importlib.reload(dataset_utils)
from utils import *
from dataset_utils import *

In [4]:
# iterate through all csv files in data folder
for file in os.listdir('data'):
    if not file.endswith('.csv'):
        continue
    df = pd.read_csv('data/'+file)
    print('\n','#'*10, file, '#'*10)
    # print column names
    print(df.columns)


 ########## cities.csv ##########
Index(['statement', 'label', 'city', 'country', 'correct_country'], dtype='object')

 ########## cities_cities_conj.csv ##########
Index(['statement', 'label', 'statement1', 'label1', 'city1', 'country1',
       'correct_country1', 'statement2', 'label2', 'city2', 'country2',
       'correct_country2'],
      dtype='object')

 ########## cities_cities_disj.csv ##########
Index(['statement', 'label', 'statement1', 'label1', 'city1', 'country1',
       'correct_country1', 'statement2', 'label2', 'city2', 'country2',
       'correct_country2'],
      dtype='object')

 ########## common_claim.csv ##########
Index(['Unnamed: 0', 'examples', 'label', 'agreement'], dtype='object')

 ########## common_claim_true_false.csv ##########
Index(['statement', 'label'], dtype='object')

 ########## companies_true_false.csv ##########
Index(['statement', 'label'], dtype='object')

 ########## counterfact_true_false.csv ##########
Index(['statement', 'label', 'relation


 ########## geonames.csv ##########
Index(['Geoname ID', 'Name', 'ASCII Name', 'Alternate Names', 'Feature Class',
       'Feature Code', 'Country Code', 'Country name EN', 'Country Code 2',
       'Admin1 Code', 'Admin2 Code', 'Admin3 Code', 'Admin4 Code',
       'Population', 'Elevation', 'DIgital Elevation Model', 'Timezone',
       'Modification date', 'LABEL EN', 'Coordinates'],
      dtype='object')

 ########## larger_than.csv ##########
Index(['statement', 'label', 'n1', 'n2', 'diff', 'abs_diff'], dtype='object')

 ########## likely.csv ##########
Index(['statement', 'label', 'likelihood'], dtype='object')

 ########## likely_old.csv ##########
Index(['statement', 'label', 'likelihood'], dtype='object')

 ########## neg_cities.csv ##########
Index(['statement', 'label', 'city', 'country', 'correct_country'], dtype='object')

 ########## neg_sp_en_trans.csv ##########
Index(['statement', 'label'], dtype='object')

 ########## smaller_than.csv ##########
Index(['statement', 'lab

In [5]:
dataset_name = "cities"
df = pd.read_csv('data/'+dataset_name+'.csv')

probe_dataset = {'dataset_name': dataset_name,
           'org_data': list(df.statement),
           'label': list(df.label)}

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
precision = torch.float16
# load model
# model_name = "mistralai/Mistral-7B-v0.1"
# model_name = "HuggingFaceH4/zephyr-7b-beta" 
model_name = "meta-llama/Llama-2-7b-chat-hf"
short_model_name = model_name.split("/")[-1]
plots_folder = f'plots/{short_model_name}'

if not os.path.exists(plots_folder):
    os.makedirs(plots_folder)
# model_name = "huggyllama/llama-7b"
# load tokenizer
if model_name == "meta-llama/Llama-2-7b-chat-hf":
    access_token = input("Enter your access token: ")
    model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=precision, token=access_token).to(device).eval()
    tokenizer = AutoTokenizer.from_pretrained(model_name, token=access_token)
    tokenizer.padding_side = "left"
    tokenizer.pad_token_id = tokenizer.eos_token_id

    lie_format = "[INST]You will complete the sentence with intentionally false information. [/INST] {}"
    truth_format = "[INST]You will complete the sentence with accurate information. [/INST] {}"

    change_format(dataset, lie_format, truth_format)

else:

    model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16).to(device).eval()
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.padding_side = "left"
    tokenizer.pad_token_id = tokenizer.eos_token_id



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

NameError: name 'dataset' is not defined

In [8]:
torch.cuda.empty_cache()
# get internal activations
module_names = [f'model.layers.{i}' for i in range(model.config.num_hidden_layers)]
num_modules = len(module_names)
token_positions = -1
batch_size = 24
# returns a dictionary with the hidden states of token_position (shape [len(selected_data), hidden_dim]) for each module
probe_dataset['hidden_states'] = get_hidden(model, tokenizer, module_names, probe_dataset['org_data'], batch_size=batch_size, token_position=-1)

100%|██████████| 63/63 [06:25<00:00,  6.12s/it]


In [9]:
probe_dataset['hidden_states']

tensor([[[ 2.5391e-02, -1.6312e-02, -1.3580e-02,  ...,  9.1705e-03,
           4.7607e-03, -7.3318e-03],
         [ 2.4948e-02, -1.4709e-02, -1.3458e-02,  ...,  9.9792e-03,
           5.7831e-03, -8.2474e-03],
         [ 2.8351e-02, -2.2003e-02, -2.1454e-02,  ...,  1.2619e-02,
           1.2482e-02, -4.8103e-03],
         ...,
         [ 2.3346e-02, -1.1665e-02, -8.3084e-03,  ...,  5.3024e-03,
           2.4490e-03, -6.5918e-03],
         [ 2.7985e-02, -2.0187e-02, -1.9791e-02,  ...,  1.1147e-02,
           6.8436e-03, -1.4145e-02],
         [ 2.9144e-02, -2.0294e-02, -2.0370e-02,  ...,  1.1719e-02,
           9.3307e-03, -1.0040e-02]],

        [[-8.7500e-01,  2.7905e-01, -1.1676e-01,  ...,  1.1494e+00,
           1.2881e+00, -5.7666e-01],
         [-8.8184e-01,  2.7979e-01, -1.2372e-01,  ...,  1.1699e+00,
           1.3018e+00, -5.7910e-01],
         [-8.6426e-01,  2.6025e-01, -1.4832e-01,  ...,  1.1533e+00,
           1.3115e+00, -5.4834e-01],
         ...,
         [-9.0088e-01,  2

In [10]:
probe_dataset['hidden_states'].to(device).shape

torch.Size([32, 1496, 4096])

In [11]:
# make test train split with equal number of labels=0 and labels=1
pos_indices = np.where(np.array(probe_dataset['label']) == 1)[0]
neg_indices = np.where(np.array(probe_dataset['label']) == 0)[0]

np.random.seed(0)
np.random.shuffle(pos_indices)
np.random.shuffle(neg_indices)

train_perc = 0.8
train_pos_indices = pos_indices[:int(train_perc*len(pos_indices))]
train_neg_indices = neg_indices[:int(train_perc*len(neg_indices))]
train_indices = np.concatenate([train_pos_indices, train_neg_indices])
np.random.shuffle(train_indices)

test_pos_indices = pos_indices[int(train_perc*len(pos_indices)):]
test_neg_indices = neg_indices[int(train_perc*len(neg_indices)):]
test_indices = np.concatenate([test_pos_indices, test_neg_indices])
np.random.shuffle(test_indices)


num_layers = probe_dataset['hidden_states'].shape[0]
train_data = probe_dataset['hidden_states'][:, train_indices, :].to(device)
train_labels = [probe_dataset['label'][i] for i in train_indices]
# convert labels to tensor
train_labels = torch.FloatTensor(train_labels).to(device).to(precision)

test_data = probe_dataset['hidden_states'][:, test_indices, :].to(device)
test_labels = [probe_dataset['label'][i] for i in test_indices]
test_labels = torch.FloatTensor(test_labels).to(device).to(precision)

In [ ]:
train_data.shape, test_data.shape

(torch.Size([32, 1196, 4096]), torch.Size([32, 300, 4096]))

In [ ]:
probes = {}

for module in module_names:
    probes[module] = LRProbe(model.config.hidden_size, device=device, precision=torch.float16)
# train probes
batch_size = 32 
num_epochs = 5
lr = 0.01
for idx, module in enumerate(module_names):
    print(f"Training probe for {module}")
    probes[module].train(train_data[idx], train_labels)



Training probe for model.layers.0


  0%|          | 0/1000 [00:00<?, ?it/s]
../aten/src/ATen/native/cuda/Loss.cu:94: operator(): block: [0,0,0], thread: [0,0,0] Assertion `input_val >= zero && input_val <= one` failed.
../aten/src/ATen/native/cuda/Loss.cu:94: operator(): block: [0,0,0], thread: [1,0,0] Assertion `input_val >= zero && input_val <= one` failed.
../aten/src/ATen/native/cuda/Loss.cu:94: operator(): block: [0,0,0], thread: [2,0,0] Assertion `input_val >= zero && input_val <= one` failed.
../aten/src/ATen/native/cuda/Loss.cu:94: operator(): block: [0,0,0], thread: [3,0,0] Assertion `input_val >= zero && input_val <= one` failed.
../aten/src/ATen/native/cuda/Loss.cu:94: operator(): block: [0,0,0], thread: [4,0,0] Assertion `input_val >= zero && input_val <= one` failed.
../aten/src/ATen/native/cuda/Loss.cu:94: operator(): block: [0,0,0], thread: [5,0,0] Assertion `input_val >= zero && input_val <= one` failed.
../aten/src/ATen/native/cuda/Loss.cu:94: operator(): block: [0,0,0], thread: [6,0,0] Assertion `input

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
train_labels

tensor([0, 0, 0,  ..., 1, 1, 0], device='cuda:0')

In [ ]:
net = torch.nn.Sequential(
            torch.nn.Linear(4, 1, bias=False),
            torch.nn.Sigmoid()
        ).to(device).to(precision)

In [ ]:
train_data.device

device(type='cpu')

In [ ]:
test_labels.dtype

torch.int64